In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
df0 = pd.read_csv('results (0M 75).csv')
df1 = pd.read_csv('results (1M 75).csv')
df2 = pd.read_csv('results (2M 75).csv')
df3 = pd.read_csv('results (3M 75).csv')
#df_list = [df0, df1, df2, df3]

In [3]:
def prepper(df):
    
    df = df.iloc[: , 1:]
    df = df[['RunId','Step','Agents','malevolent']]
    
    return df

In [4]:
def organizer(df):
    grouped_df = df.groupby("RunId")
    groups = []
    
    for group in df["RunId"].unique():
      group_df = grouped_df.get_group(group).iloc[:, 1:3]
      group_df = group_df.set_index(group_df["Step"].values).T.reset_index(drop = True).iloc[1:]
      malevolent = grouped_df.get_group(group)["malevolent"].sum()
      group_df.insert(column = "RunId", loc = 0, value = [group])
      group_df.insert(column = "malevolent", loc = group_df.shape[1], value = [malevolent])
      groups.append(group_df)

    final_df = pd.concat(groups).set_index("RunId")
    df1 = final_df.pop('malevolent')
    final_df['malevolent']=df1
    
    return final_df

In [5]:
df0 = prepper(df0)
df1 = prepper(df1)
df2 = prepper(df2)
df3 = prepper(df3)

In [6]:
def organizer(df):
    grouped_df = df.groupby("RunId")
    groups = []
    
    for group in df["RunId"].unique():
      group_df = grouped_df.get_group(group).iloc[:, 1:3]
      group_df = group_df.set_index(group_df["Step"].values).T.reset_index(drop = True).iloc[1:]
      malevolent = grouped_df.get_group(group)["malevolent"].sum()
      group_df.insert(column = "RunId", loc = 0, value = [group])
      group_df.insert(column = "malevolent", loc = group_df.shape[1], value = [malevolent])
      groups.append(group_df)

    final_df = pd.concat(groups).set_index("RunId")
    df1 = final_df.pop('malevolent')
    final_df['malevolent']=df1
    
    return final_df

In [7]:
df0 = organizer(df0)
df1 = organizer(df1)
df2 = organizer(df2)
df3 = organizer(df3)

In [8]:
frames = [df0, df1, df2, df3]
df = pd.concat(frames)

In [9]:
dft = df.pop('malevolent')
df['malevolent']=dft
df.loc[df["malevolent"] > 0, "malevolent"] = 1

In [10]:
df

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,malevolent
RunId,,,,,,,,,,,,,,,,,,,,,
0,259,253,251,247,244,241,239,237,234,232,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,282,280,276,272,269,265,261,258,256,253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,267,261,259,256,252,249,246,243,241,238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,257,251,250,246,244,242,239,235,234,230,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,253,249,246,244,241,239,235,233,229,226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,259,252,249,245,245,241,238,234,232,228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
71,262,258,256,252,251,248,246,243,240,236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
72,265,260,256,254,251,248,246,242,240,236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


<b>XGBoost Work</b>

In [16]:
X, y = df.iloc[:,:-1],df.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [12]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                          max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

[09:17:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [13]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.424026


In [14]:
xg_reg.score(X, y)

0.07414973935043234

In [15]:
model = xgb.XGBClassifier()
kfold = KFold(n_splits=50)
results = cross_val_score(model, X, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 66.33% (32.40%)
